In [ ]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
drive.mount("/content/gdrive")

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

"""
Create Your Own Quantum Mechanics Simulation (With Python)
Philip Mocz (2020) Princeton Univeristy, @PMocz
Simulate the Schrodinger-Poisson system with the Spectral method
"""

V_hist = []
psi_hist = []
t_hist = []


"""Quantum Simulation"""

#Simulation Parameters
N = 512 #Spatial Resolution
t = 0 #Current time of simulation
tEnd = 0.03 #Time at which simulation ends
dt = 0.0001 #Timestep
tOut = 0.0001 #Draw Frequency
G = 4000 #Gravitational Constant
plotRealTime = True #Switch on for plotting as simulation goes along

#Domain [0,1] x [0,1]

L = 1
xlin = np.linspace(0, L, num=N+1) #Note: x=0 and x=1 are the same point
xlin = xlin[0:N] #Chop off periodic point
xx, yy = np.meshgrid(xlin, xlin)

#Initial Condition
amp = 0.01
sigma = 0.03
rho = 0.9
rho+= 2*amp*np.exp(-((xx-0.5)**2+(yy-0.5)**2)/2/sigma**2)/(sigma**3*np.sqrt(2*np.pi)**2)
rho+= 1.5*amp*np.exp(-((xx-0.2)**2+(yy-0.7)**2)/2/sigma**2)/(sigma**3*np.sqrt(2*np.pi)**2)
rho+= amp*np.exp(-((xx-0.4)**2+(yy-0.6)**2)/2/sigma**2)/(sigma**3*np.sqrt(2*np.pi)**2)
rho+= amp*np.exp(-((xx-0.6)**2+(yy-0.8)**2)/2/sigma**2)/(sigma**3*np.sqrt(2*np.pi)**2)
rho+= amp*np.exp(-((xx-0.8)**2+(yy-0.2)**2)/2/sigma**2)/(sigma**3*np.sqrt(2*np.pi)**2)
rho+= amp*np.exp(-((xx-0.6)**2+(yy-0.7)**2)/2/sigma**2)/(sigma**3*np.sqrt(2*np.pi)**2)
rho+= amp*np.exp(-((xx-0.7)**2+(yy-0.4)**2)/2/sigma**2)/(sigma**3*np.sqrt(2*np.pi)**2)
rho+= amp*np.exp(-((xx-0.3)**2+(yy-0.3)**2)/2/sigma**2)/(sigma**3*np.sqrt(2*np.pi)**2)

#Normalise wavefunction to <|psi|^2> = 1
rhobar = np.mean(rho)
rho /= rhobar
psi = np.sqrt(rho)

np.save('/content/gdrive/My Drive/psi/psi_0.npy', psi, allow_pickle=True)

#Fourier Space Variables
klin = 2.0 * np.pi / L * np.arange(-N/2, N/2)
kx, ky = np.meshgrid(klin, klin)
kx = np.fft.ifftshift(kx)
ky = np.fft.ifftshift(ky)
kSq = kx**2 + ky**2

#Potential
Vhat = -np.fft.fftn(4.0*np.pi*G*(np.abs(psi)**2-1.0)) / ( kSq  + (kSq==0))
V = np.real(np.fft.ifftn(Vhat))

#Number of timesteps
Nt = int(np.ceil(tEnd/dt))

psi_hist = psi_hist + [psi]
V_hist = V_hist + [V]
t_hist = t_hist + [t]

#Simulation Main Loop
for i in range(Nt):

  #prep figure
  fig = plt.figure(figsize=(6,4), dpi=200)
  grid = plt.GridSpec(1, 2, wspace=0.3, hspace=0.0)
  ax1 = plt.subplot(grid[0,0])
  ax2 = plt.subplot(grid[0,1])
  outputCount = 1

  # (1/2) kick
  psi = np.exp(-1.j*dt/2.0*V) * psi

  #drift
  psihat = np.fft.fftn(psi)
  psihat = np.exp(dt * (-1.j*kSq/2.))  * psihat
  psi = np.fft.ifftn(psihat)

  #Update Potential
  Vhat = -np.fft.fftn(4.0*np.pi*G*(np.abs(psi)**2-1.0)) / ( kSq  + (kSq==0))
  V = np.real(np.fft.ifftn(Vhat))
  V_hist = V_hist + [V]

  # (1/2) kick
  psi = np.exp(-1.j*dt/2.0*V) * psi
  psi_hist = psi_hist + [psi]

  #Update time
  t += dt
  t_hist = t_hist + [t]

  print(np.round(t,4))

  if np.round(t, 4) % 0.0075 == 0:
    np.save('/content/gdrive/My Drive/psi/psi_' + str(t) + '.npy', psi, allow_pickle=True)

  #Plot in real time
  plotThisTurn = False

  if t + dt > outputCount*tOut:
    plotThisTurn = True

  if (plotRealTime and plotThisTurn) or (i == Nt-1):

    plt.sca(ax1)
    plt.cla()
    plt.imshow(np.log10(np.abs(psi)**2), cmap = 'viridis')
    plt.colorbar(shrink = 0.5)
    plt.clim(-1,2)
    plt.title(r'$\log_{10}(|\psi|^2)$')
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)
    ax1.set_aspect('equal')

    plt.sca(ax2)
    plt.cla()
    plt.imshow(np.angle(psi), cmap = 'viridis')
    plt.colorbar(shrink = 0.5)
    plt.clim(-np.pi, np.pi)
    plt.title(r'${\rm Angle}(\psi)$')
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    ax2.set_aspect('equal')

    plt.pause(0.001)
    outputCount += 1

In [ ]:
V_hist_np = np.array(V_hist)
np.save('/content/gdrive/My Drive/V_hist.npy', V_hist_np, allow_pickle=True)

t_hist_np = np.array(t_hist)
np.save('/content/gdrive/My Drive/t_hist_qs.npy', t_hist_np, allow_pickle=True)

psi_hist_np = np.array(psi_hist)
np.save('/content/gdrive/My Drive/psi_hist.npy', psi_hist_np, allow_pickle=True)